# Table of Contents

1. Stationarity
2. Differencing
3. Machine Learning
    1. Linear Models
    2. Deep Learning
4. AutoML with AutoGluon

In [ ]:
# imports
from random import random
from numpy import transpose, matrix, sin, mean
from numpy.linalg import lstsq
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# Stationarity

* $\mu$ is constant
* $\sigma$ is constant
* No seasonality

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 6), dpi=80)
n = 100

# mean is constant but variance varies
y = [random()*x if x%2 else (-1)*random()*x for x in list(range(n))[::-1]]
ax1.plot(y)
ax1.plot([0]*n)

# variance is constant but mean is increasing
y = [2 + 5*x + 25 * random() if x%2 else 2 + 5*x - 25 * random() for x in range(n)]
ax2.plot(y)
ax2.plot([2 + 5*x for x in range(n)])

y = [sin(x/4) for x in range(n)]
ax3.plot(y)

# Differencing

Differencing is a technique to transform a non-stationary time series into a stationary one. It involves subtracting the current value of the series from the previous one, or from a lagged value.

In [ ]:
y = [2 + 5*x + 25 * random() if x%2 else 2 + 5*x - 25 * random() for x in range(n)]
y_diff = [y[x+1] - y[x]  for x in range(n-1)]

plt.plot(y)
plt.plot(y_diff)

# Machine Learning

## Linear Models

### Autoregressive Model - $AR(p)$

$y_i = \phi_0 + \phi_1 y_{i-1}+ \phi_2 y_{i-2}+ \cdots + \phi_p y_{i-p} + \epsilon_i$

$\iff$

$\epsilon_i = y_i - \phi_0 - \sum_{j=1}^{p}\phi_jy_{i-j}$

Our goal is to minimize

$\sum_{i=p+1}^{n}\epsilon^2 = \sum_{i=p+1}^{n} (y_i - \phi_0 - \sum_{j=1}^{p}\phi_jy_{i-j})^2$

In [ ]:
# dummy data
n = 100
y = [2 + 5*sin(x) + 2*random()  for x in range(n)]
plt.plot(y)

In [ ]:
# show p=1 (manual)
p = 1
X = transpose(matrix([[1]*(len(y) - p), y[:-1]]))
Y = transpose(matrix(y[p:]))
coef = lstsq(X, Y, rcond=None)[0]
print(f"Coefficients: {[coef[0,0], coef[1,0]]}")
y_hat = [coef[0, 0]] + [coef[0, 0] + coef[1, 0]*x for x in y[:-1]]
plt.plot(y)
plt.plot(y_hat)

In [ ]:
# how to pick to p: ACF and PACF plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), dpi=80)
plot_acf(y, ax=ax1, lags=50)
plot_pacf(y, ax=ax2, lags=50)

plt.show()

In [ ]:
# show plot when p=3
model = AutoReg(y, lags=3)
model_fit = model.fit()
print('Coefficients: %s' % model_fit.params)
y_hat = model_fit.predict()
plt.plot(y)
plt.plot(y_hat)

### Moving Average Model - $MA(q)$

$y_i = \theta_0 + \epsilon_i + \theta_1 \epsilon_{i-1}+ \theta_2 \epsilon_{i-2}+ \cdots + \theta_q \epsilon_{i-q}$

_In the MA model, rather than using the earlier values of a variable for prediction, it uses earlier prediction error terms which result when regressing a series from its past values._

In [ ]:
# perform regression
X = transpose(matrix([[1]*len(y), list(range(n))]))
Y = transpose(matrix(y))
coef = lstsq(X, Y, rcond=None)[0]
y_hat = [coef[0, 0] + coef[1, 0]*x for x in y]
plt.plot(y)
plt.plot(y_hat)

# use the erros from regression to find the MA coefficients
q = 1
errors = [y[x] - y_hat[x] for x in range(n)]
X = transpose(matrix([[1]*(len(errors) - q), errors[:-1]]))
Y = transpose(matrix(y[q:]))
coef = lstsq(X, Y, rcond=None)[0]
print(f"Coefficients: {[coef[0,0], coef[1,0]]}")
y_hat = [coef[0, 0] + coef[1, 0]*x for x in y]
plt.plot(y)
plt.plot(y_hat)

In [ ]:
mean(y)

### $ARMA(p, q)$

*AR and MA can be used together to form a new model called the Autoregressive Moving Average (ARMA)*

$y_i = \phi_0 + \phi_1 y_{i-1}+ \phi_2 y_{i-2}+ \cdots + \phi_p y_{i-p} + \theta_0 + \theta_1 \epsilon_{i-1}+ \theta_2 \epsilon_{i-2}+ \cdots + \theta_q \epsilon_{i-q} + \epsilon_i$

### $ARIMA(p, q, d)$

$d$ *refers to the number of differencing transformations required by the time-series to attain stationarity.*

## Deep Learning Models

* Recurrent Neural Networks (RNN)
* Gated Recurrent Unit (GRU)
* Long Short-Term Memory (LSTM)
* Indepedently Recurrent Neural Networks (IndRNN)

# AutoML

* Model Selection
* Hyperparameter Optimization
* Feature Engineering

## AutoGluon

Visit the quickstart guide [here](https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-quick-start.html).

In [ ]:
# Prep timeseries dataframe
item_id = "H1"
prediction_length = 24

x = list(map(lambda x: x*3.6*10**12, range(n)))
x_train = x[:-prediction_length]
x_test = x[-prediction_length:]
y_train = y[:-prediction_length]
y_test = y[-prediction_length:]

df_train = pd.DataFrame({"timestamp": x_train, "target": y_train, "item_id": [item_id]*(n-prediction_length)})
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column="item_id",
    timestamp_column="timestamp"
)
df_test = pd.DataFrame({"timestamp": x_test, "target": y_test, "item_id": [item_id]*prediction_length})
test_data = TimeSeriesDataFrame.from_data_frame(
    df_test,
    id_column="item_id",
    timestamp_column="timestamp"
)
train_data.tail()

In [ ]:
# train
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="target",
    eval_metric="MSE",
)
predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=1200,
)

In [ ]:
predictor.leaderboard(train_data)


In [ ]:
plt.figure(figsize=(20, 3))
predictions = predictor.predict(train_data)
y_past = train_data.loc[item_id]["target"]
y_pred = predictions.loc[item_id]
y_test = test_data.loc[item_id]["target"]

plt.plot(y_past, label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")
plt.plot(y_test, label="Future time series values")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
)
plt.legend()